<div align="center">

<font size="5">

Laboratorium z przedmiotu: \
**Głębokie uczenie i analiza obrazów**

Ćwiczenie 6: \
**Rekurencyjne sieci neuronowe**

</font>

\
Marta Szarmach \
Zakład Telekomunikacji Morskiej \
Wydział Elektryczny \
Uniwersytet Morski w Gdyni

11.2023
</div>



# 1. Wprowadzenie

**Rekurencyjne sieci neuronowe** (ang. *Recurrent Neural Networks*, RNN) to takie sieci neuronowe, które dokonują w danej chwili $t$ predykcji nie tylko na podstawie ,,obecnych'' danych ($x_t$), ale też ,,wcześniejszych' (na podstawie zawartości **ukrytego stanu** $h_{t-1}$ zależącego od danych z wcześniejszych chwil $x_{t-1}$):
\begin{equation*}
    h_t = f(h_{t-1},x_t) = \textrm{tanh} \left( \Theta_h h_{t-1} + \Theta_x x_t + b \right)
\end{equation*}

To powoduje, że rekurencyjne sieci neuronowe świetnie sprawdzają się w analizie **sekwencji danych**, tj. danych składających się z elementów z różnych kroków czasowych, np.: fragmentów filmu (składające się z następujących po sobie ramek), zdań (składające się z następujących po sobie wyrazów), wycinków mowy czy muzyki. Modele te mogą:
* analizować wejściową sekwencję danych i zwracać pojedynczą wartość (modele *many-to-one*), np. analizować sentyment tekstu, 
* analizować wejściową sekwencję danych i zwracać inną sekwencję (modele *many-to-many*, inaczej nazywane seq2seq), np. dokonywać tłumaczenia zdań, 
* na podstawie pojedynczej danej wejściowej generować całkowicie nowe sekwencje (modele *one-to-many*), np. generować słowny opis do zdjęcia.

<div align="center">

<img src='https://raw.githubusercontent.com/Argenni/GUiAO_lab/c2ecb6dd39a2ca46e3feff4fe513c09d36bc0270/rys/10_seq2seq.png'/>

<font size="1">Przykład modelu *many-to-many*. Grafika: Justin Johnson, University of Michigan</font>
</div>

Klasyczne sieci rekurencyjne borykają się jednak z pewnymi problemami. Mimo że formalnie nie posiadają ograniczeń co do długości analizowanej sekwencji danych, w rzeczywistości analiza długich sekwencji sprawia im problemy (występują takie zjawiska, jak *vanishing/exploding gradient*), modele gubią szerszy kontekst sekwencji. Aby sobie z tym radzić, proponuje się udoskonalenie klasycznych RNNów o następujące mechanizmy:
* **context vector** $c_t$ - dodatkowa pamięć (mogąca różnić się od *hidden state*), przechowująca szerszy kontekst (np. informację o płci podmiotu w generowanym zdaniu), dopóki jest on potrzebny,
* **bramki** (ang. *gates*) - współczynniki modyfikujące sposób wyznaczania $h_t$:
    * *update gate* - decyduje, jak silnie opierać się na dotychczasowych danych podczas uaktualniania *hidden state* $h_t$ lub $c_t$,
    * *forget gate* lub *reset gate* - decyduje, które dane z przeszłości zapomnieć (czy wyzerować $c_t$ lub $h_t$),
    * *output gate* - decyduje, jak silnie opierać się na kontekście $c_t$ przy uaktualnianiu $h_t$.

Przykładem zmodyfikowanej sieci RNN są sieci **GRU** (ang. *Gated Recurrent Unit*, wykorzystujące *update gate* i *reset gate*) oraz **LSTM** (ang. *Long Short-Term Memory*, wykorzystujące wszystkie bramki oraz pamięć o szerszym kontekście $c_t$).

<div align="center">

<img src='https://raw.githubusercontent.com/Argenni/GUiAO_lab/main/rys/11_gru%26lstm.png'/>

<font size="1">Grafika: medium.com</font>
</div>

Innym udoskonaleniem modeli opartych na RNNach było wprowadzenie **atencji**, czyli mechanizmu decydującego o tym, które z danych z przeszłości mają większy wpływ na dokonanie predykcji w obecnym kroku (np. podczas dokonywania tłumaczenia zdania, większa uwaga poświęcana była wyrazowi aktualnie tłumaczonemu, nawet przy zmianie szyku zdania). Wprowadzenie mechanizmu atencji spowodowało powstanie niezwykle silnych modeli (takich jak BERT czy GPT), opartych na **transformerach**, realizujących zadania z zakresu **przetwarzania języka naturalnego** (obejmujące analizę tekstu i mowy ludzkiej), takie jak m.in.:
* tłumaczenie zdań z jednego języka w drugi,
* uzupełnianie brakujących słów/liter w tekście, generacja tekstu,
* rozpoznawanie mowy i tekstu (OCR),
* streszczanie długich tekstów, wyszukiwanie sentymentu.


# 2. Cel ćwiczenia

**Celem niniejszego ćwiczenia** jest zapoznanie się z działaniem rekurencyjnych sieci neuronowych poprzez implementację prostej sieci CharRNN, służącej do generacji tekstu znak po znaku, z wykorzystaniem języka Python i biblioteki PyTorch.


# 3. Stanowisko laboratoryjne

Do wykonania niniejszego ćwiczenia niezbędne jest stanowisko laboratoryjne, składające się z komputera klasy PC z zainstalowanym oprogramowaniem:
* językiem programowania Python (w wersji 3.8),
* IDE obsługującym pliki Jupyter Notebook (np. Visual Studio Code z rozszerzeniem ipykernel).


# 4. Przebieg ćwiczenia
## Generacja nowych łacińskich nazw roślin z wykorzystaniem CharRNN w PyTorch

W ramach tego ćwiczenia, silnie opierać się będziemy na kodzie z [TEGO](https://github.com/nikhilbarhate99/Char-RNN-PyTorch/blob/master/CharRNN.py) repozytorium. Pobawimy się trochę w biologów/lingwistów: naszym zadaniem będzie zaprojektowanie rekurencyjnej sieci neuronowej, która nauczy się generacji łacińskich nazw dla nowoodkrytych gatunków roślin! Istniejące nazwy, na których nasz model się nauczy, pobierzemy ze zbioru danych UCI Plants [(TUTAJ)](https://archive.ics.uci.edu/dataset/180/plants) (<font size=2>Hamalinen,W.. (2008). Plants. UCI Machine Learning Repository. https://doi.org/10.24432/C5HS40.</font>)

Na początku wykonaj poniższy fragment kodu, aby zaimportować biblioteki niezbędne do wykonania poniższego ćwiczenia:
* **NumPy** - biblioteka umożliwiająca wykonywanie wysoko zoptymalizowanych obliczeń matematycznych na objektach typu *numpy array* (wielowymiarowych tablic),
* **PyTorch** - biblioteka wspomagająca budowanie architektur sieci neuronowych, posiadająca wbudowane moduły odpowiadające różnym warstwom sieci neuronowych, automatyczne obliczanie gradientów (*autograd*) niezbędne do przeprowadzenia treningu sieci neuronowych,
* **wget** - biblioteka umożliwiająca pobieranie plików z zewnętrznych źródeł (np. stron www) oraz **os** - biblioteka umożliwiająca zarządzanie tymi plikami z poziomu systemu operacyjnego.

In [2]:
# ! python -m pip install numpy==1.22.3
# ! python -m pip install torch==2.0.1
# ! python -m pip install wget==3.2

import numpy as np
import torch
import wget
import os

### Wczytanie i przygotowanie danych

Na wstępie musimy przygotować odpowiednio dane, na których będziemy pracowali. W pierwszej części przygotowanego przeze mnie kodu dokonuje się pobranie i wczytanie pliku `plants.data`, zawierającego dane z zestawu Plants: najpierw dane wczytywane są linijka po linijce, a następnie przekształcane do tensora składającego się z ciągu znaków (przykładowo, nazwa pierwszej rośliny, *abelia*, widoczna jest jako lista `['a','b','e','l','i','a']`; po każdej nazwie sztucznie dodaję znak nowej linii `\n`).  

Musimy pamiętać, że komputer operuje na liczbach, a nie znakach alfanumerycznych. Musimy zatem przekształcić znaki do postaci liczb. Kolejnym ważnym krokiem w przygotowaniu danych jest zatem stworzenie dwóch słowników:
* `char_to_num`, zawierającego powiązania pomiędzy znakami a przypisanymi im liczbami-indeksami (np. `{'a':4}`)
* `num_to_char`, zawierającego odwrotne powiązania, tj. indeks:znak (np. `{4:'a'}`). 

<font size=2>Aby to zrobić, wystarczy przeiterować po wszystkich elementach listy `characters`, zapamiętując analizowany element listy oraz jego indeks (w tym celu warto przekształcić listę do formy `enumerate` ([TUTAJ](https://www.geeksforgeeks.org/enumerate-in-python/))).</font>

Po utworzeniu tych słowników, przeiteruj po każdym znaku ze zbioru `input_text`, aby przekonwertować dane wejściowe ze znaków na ich indeksy! Po uzupełnieniu i uruchomieniu poniższego kodu, powinieneś widzieć efekty tego przekształcenia (od teraz dla naszego modelu, *abelia* to `[4,5,8,15,12,4]`). Od razu przeksztłać otrzymaną listę w  `torch.tensor`.

In [13]:
# ------------------------- Inicjalizacja -----------------------
# Pobierz i wczytaj plik z tekstem - na razie linijka po linijce
if not os.path.exists("utils/plants.data"):
    wget.download("https://raw.githubusercontent.com/Argenni/GUiAO_lab/main/utils/plants.data", out="utils/plants.data")
with open("utils/plants.data", "r") as f:
    data_raw = [s.strip() for s in f.readlines()]
# Zapisz wczytany tekst w formie pojedynczych znaków
data_lines = []
for data_line in data_raw: # Dla każdej wczytanej linijki tekstu:
    data_split = data_line.split(",") # wyodrębnij tylko kolumnę z łacińskimi nazwami roślin
    data_lines.append([*data_split[0]+"\n"]) # rozbij tekst na pojedyncze znaki i dodaj na końcu znak końca linii 
input_text = sum(data_lines,[]) # zapisz w formie spójnej listy
print("Wczytano tekst; przykładowy fragment: ")
print(input_text[0:20])
# Wyodrębnij pojedyncze znaki (utwórz zbiór characters ze znakami)
characters = sorted(set(input_text))
print("Pojedynczych znaków: "+str(len(characters)))

# ----------------------- UZUPEŁNIJ KOD ----------------------------------
# Każdemu znakowi przypisz jego reprezentację numeryczną (i na odwrót) - utwórz dwa słowniki
char_to_num = { char:index for index,char in enumerate(characters) }
num_to_char = { index:char for index,char in enumerate(characters) }
# Zakoduj całość wczytanego tekstu i zapisz jako torch.tensor
embedded_text = [char_to_num[char] for index,char in enumerate(input_text)]
embedded_text = torch.tensor(embedded_text)
# -----------------------------------------------------------------------

print("Fragment tekstu po zakodowaniu: ")
print(embedded_text[0:20])
embedded_text = torch.unsqueeze(embedded_text, dim=1)

Wczytano tekst; przykładowy fragment: 
['a', 'b', 'e', 'l', 'i', 'a', '\n', 'a', 'b', 'e', 'l', 'i', 'a', ' ', 'x', ' ', 'g', 'r', 'a', 'n']
Pojedynczych znaków: 30
Fragment tekstu po zakodowaniu: 
tensor([ 4,  5,  8, 15, 12,  4,  0,  4,  5,  8, 15, 12,  4,  1, 27,  1, 10, 21,
         4, 17])


### Architektura sieci neuronowej

Czas zaprojektować architekturę naszej rekurencyjnej sieci neuronowej - stworzymy ją z wykorzystaniem  biblioteki PyTorch, w klasie o nazwie `RNN`. Tym razem nasza architektura będzie stosunkowo prosta: zbudujemy są z trzech PyTorchowych bloków.
* Pierwszym blokiem - `self.encoder` - będzie `nn.Embedding` ([TUTAJ](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)), która dokonuje zamiany wejściowych danych (indeksów znaków) w postaci zwykłego skalara do zakodowanego wektora - przypomnijmy sobie przekształcenie *one-hot-encoding*. Musimy podać tej warstwie informacje o ilości kodowanych liczb (`num_embeddings`) oraz oczekiwanej długości wektora po zakodowaniu (`embedding_dim`) - w tym przypadku, oba te argumenty muszą być równe i wynosić ilość pojedynczych znaków występujących w naszym tekście (u nas jest to zmienna `input_size`).
* Drugim blokiem - `self.lstm` - jest sieć LSTM `nn.LSTM` ([TUTAJ](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)) - to ją będziemy trenować. Musimy jej podać takie dane, jak: wymiarowość danych (`input_size`), wielkość *hidden state* (`hidden_size`) oraz ilość warstw, z ilu składa się nasza LSTM (`num_layers`) - tę wartość ustawmy na stałe na 3.
* Ostatnim blokiem jest warstwa liniowa `nn.Linear` - wejściowych neuronów powinno być tyle, ile wynosi wymiarowość *hidden_state*, a wyjściowych - tyle, ile wynosi znaków w zbiorze danych (by każdy wyjściowy neuron mógł oszacować prawdopodobieństwo, że dany znak ma być tym następnym) - jest to wartość przechowywana w argumencie `output_size`.

Uzupełnij zatem metody `__init__` i `forward` klasy RNN. Układając przepływ danych, pamiętaj, że sieć LSTM oprócz przyjmowania danych wejściowych `X` i zwracaniu `output`, przyjmuje też i zwraca zawartość *hidden state* - `hidden_state`.

In [14]:
class RNN(torch.nn.Module):
    """
    Model rekurencyjnej sieci neuronowej: Embedding -> LSTM -> Linear
    """
    
    def __init__(self, input_size, hidden_size, output_size):
        """
        Definiuje budowę sieci. Argumenty: \n
        - input_size - wielkość tworzonych na pierwszej warstwie embeddingów
            (dla OHE równe ilości znaków w słowniku), int, skalar, \n
        - hidden_size - długość pamiętanego hidden state, int, skalar, \n
        - output_size - ilość neuronów na wyjściu warstwy liniowej, też równa ilości znaków w słowniku, int, skalar.
        """
        super().__init__()
        # ---------------------- UZUPEŁNIJ KOD -----------------------------------
        # Zacznij definiować budowę sieci:
        # Embedding layer
        self.encoder = torch.nn.Embedding(num_embeddings=input_size, embedding_dim=input_size)
        # LSTM
        self.lstm = torch.nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=3)
        # Linear
        self.decoder = torch.nn.Linear(in_features=hidden_size, out_features=output_size)
        # ---------------------------------------------------------------------

    def forward(self, X, hidden_state=None):
            """ 
            Definiuje przepływ danych w sieci. Argumenty: \n 
            - X - dane wejściowe, tj.  shape=(sequence_len, input_size), \n
            - hidden_state - (opcjonalnie) dane, którymi chcemy inicjalizować hidden_state naszej LSTM,
                torch.tensor, shape=(num_layers, batch_size, hidden_size) domyślnie None, co oznacza inicjalizację zerami. \n
            Zwraca: output - odpowiedź sieci, torch.tensor, shape=(sequence_size, batch_size, hidden_size).
            """
            # ------------------- UZUPEŁNIJ KOD ----------------
            # Zakoduj wejściową sekwencję (OHE) z wykorzystaniem warstwy Embedding
            X = self.encoder(X)
            # Przekaż zakodowaną sekwencję do LSTMa (LSTM zwraca swoje wyjście oraz zawartość hidden state)
            output, hidden_state = self.lstm(X, hidden_state)
            # Ostateczna predykcja - wyjście warstwy Linear
            output = self.decoder(output)
            # --------------------------------------------------
            return output, hidden_state

### Generacja tekstu

Ideą działania tak zdefiniowanej przez nas sieci neuronowej jest to, by na jej wyjściu otrzymywać wartości liczbowe, na podstawie których możemy określić pewien rozkład prawdopodobieństwa - każdemu znanemu znakowi przypisywane jest prawdopodobieństwo bycia następnym znakiem w generowanej sekwencji. Aby otrzymać ten rozkład, ,,znormalizujemy'' sobie wyjścia naszej sieci z pomocą funkcji `softmax`, a następnie utworzymy go, korzystając z `torch.distributions.Categorical` ([TUTAJ](https://pytorch.org/docs/stable/distributions.html#categorical)). Taki rozkład następnie próbkujemy (metodą `sample()`), aby wylosować następny znak - nie chcemy wybierać tylko znaku z najwyższym prawdopodobieństwem, bo generowane przez nas sekwencje byłyby monotonne! 

Napisz teraz funkcję `generate_text_embedding`, która dokonuje generacji ciągu indeksów znaków. 

Moje dodatkowe założenia:
* Wygenerowany tekst ma być reakcją sieci neuronowej na znak końca linii - a zatem to, co w zbiorze danych jest początkiem kolejnej łacińskiej nazwy rośliny.
* Wygenerowana sekwencja ma zawierać nie więcej niż `sequence_size` znaków i jednocześnie nie więcej niż 2 wygenerowane nazwy (tj. mogą wystąpić maksymalnie dwa znaki końca linii).

In [66]:
def generate_text_embedding(sequence_size, rnn, char_to_num):
    """
    Funkcja generująca (za pomocą stworzonego przez nas RNNa klasy RNN) tekst znak po znaku, 
    w formie ciągu numerów-indeksów znaków zgodnych ze słownikiem char_to_num. Argumenty: \n
    - sequence_size - maksymalna długość generowanej sekwencji znaków, int, skalar, \n
    - rnn - obiekt klasy RNN (zdefiniowanej przez nas wcześniej), zawierający naszą RNN, \n
    - char_to_num - słownik, wiążący symbol znaku (np. "a") z przypisanym mu numerem int (np. 0), len=len(characters). \n
    Zwraca: output_sequence - lista zawierająca indeksy kolejnych wygenerowanych znaków (zgodne z char_to_num).
    """
    rnn.eval() # przestawienie sieci neuronowej w tryb predykcji, a nie treningu
    input_sequence = torch.tensor(char_to_num["\n"]).reshape(1,-1) # niech generowana sekwencja będzie odpowiedzą na znak nowej linii
    output_sequence = [] # zmienna przechowująca wygenerowaną sekwencję znaków
    hidden_state = None # inicjalizacja hidden state (tak naprawdę) samymi zerami
    newline_counter = 0 # zliczanie ilości wygenerowanych znaków nowej linii - w celu zatrzymania generacji
    char_counter = 0 # zliczanie ilości wygenerowanych znaków - w celu zatrzymania generacji
    while (True):
        # ------------------------- UZUPEŁNIJ KOD ---------------------------------
        # Wygeneruj sekwencję znaków z wykorzystaniem naszej sieci 
        pred, hidden_state = rnn(input_sequence, hidden_state)
        # "Znormalizuj" wyjście, stosując funkcję softmax
        pred = torch.nn.functional.softmax(torch.squeeze(pred), dim=0)
        # Określ rozkład prawdopodobieństwa następnego znaku
        distribution = torch.distributions.Categorical(torch.squeeze(pred))
        # Próbkuj z tego rozkładu - to będzie nowy znak
        output = distribution.sample()
        # Zapisz indeks nowego znaku do output_sequence
        output_sequence.append(output.item())
        # -------------------------------------------------------------------------
        input_sequence[0][0]=output.item() # obecne wyjście = nowe wejście
        char_counter = char_counter + 1 # wygenerowano nowy znak, więc zwiększ licznik o 1
        if (output.item()==char_to_num["\n"]): newline_counter = newline_counter + 1 # jeśli wygenerowano znak nowej linii, zwiększ licznik
        if (newline_counter>=2): break # zatrzymaj generację, jeśli wygenerowano 2 linijki tekstu...
        if (char_counter >= sequence_size): break # lub kiedy długość sekwencji przekraczałaby sequence_size
    return output_sequence

Inna niezbędna funkcja `embedding_to_string` ma za zadanie sekwencję znaków w postaci indeksów przekształcić do formy zwykłego tekstu (stringa). Chodzi o to, by, przykładowo, z listy `[4,5,6,7]` otrzymać zrozumiały string `abcd`.

Przeiteruj zatem po wszystkich elementach pewnej sekwencji indeksów `sequence_embeddings`, odszukaj odpowiadające im znaki w słowniku `num_to_char` i zapisz w osobnej liście `gen`. W ostatniej części, zrealizowałam już łączenie takiej listy znaków w jeden spójny string.

In [67]:
def embedding_to_string(sequence_embeddings, num_to_char):
    """
    Funkcja zamieniająca wygenerowaną sekwencję w postaci listy indeksów znaków na pojedynczy string. Argumenty: \n
    - sequence_embeddings - lista zawierajaca indeksy wegenerowanych znaków (int) zgodne ze słownikiem num_to_char, \n
    - num_to_char - słownik, wiążący indeks znaku (int) (np. 0) z właściwym znakiem (np.  "a"), len=len(characters). \n
    Zwraca: text - string zawierający spójną, zdekodowaną, zrozumiałą dla człowieka wygenerowaną sekwencję znaków.
    """
    # ---------------------- UZUPENIJ KOD ------------------------
    # Utwórz listę gen, która zawiera pobrane ze słownika num_to_char znaki odpowiadające indeksom z sequence_embeddings
    gen = [num_to_char[index] for index in sequence_embeddings]
    # -----------------------------------------------------------
    text = "" # inicjalizacja zmiennej przechowującej tekst
    text = text.join(gen) # połącz elementy listy w jeden, spójny tekst
    return text

Kod z poniższej komórki pozwoli Ci sprawdzić poprawność implementacji powyższych funkcji. Sprawdź, czy bez zgłaszania błędów, próbny wyygenerowany tekst w formie indeksów będzie pewnymi liczbami naturalnymi, a próbny tekst zdekodowany z indeksów 4,5,6,7 to abcd.

In [70]:
# Próbna generacja ciągu 4 znaków
rnn = RNN(len(num_to_char), 512, len(num_to_char)) # obiekt z naszą siecią
rnn = rnn.double()
text_embedding_test = generate_text_embedding(4, rnn, char_to_num)
print("Próbny wygenerowany tekst (bez treningu sieci) w formie indeksów znaków:")
print(text_embedding_test)
# Próbne zdekonowanie ciągu 4 znaków - abcd
text_test = embedding_to_string([4,5,6,7], num_to_char)
print("Próbny tekst zdekodowany z indeksów 4,5,6,7:")
print(text_test)

Próbny wygenerowany tekst (bez treningu sieci) w formie indeksów znaków:
[17, 7, 11, 23]
Próbny tekst zdekodowany z indeksów 4,5,6,7:
abcd


### Trening rekurencyjnej sieci neuronowej

Trening naszego RNNa poprowadź podobnie, jak w przypadku dotychczasowych sieci neuronowych. Musimy przestawić sieć w tryb treningu, przeprowadzić *forward pass* z wykorzystaniem pewnej wejściowej sekwencji znaków, odebrać sekwencję wygenrowaną przez model, obliczyć wartość funkcji kosztu (użyjemy znaną Ci już CrossEntropyLoss), wyzerować dotychczasowe gradienty, przeprowadzić propagację wsteczną kosztu i zaktualizować parametry w jednym kroku algorytmu optymalizacji (znów skorzystamy z optymalizatora Adam).

Jedyną różnicą jest sposób walidacji otrzymywanych z sieci wyników. Chcemy, by model umiał generować tekst podobny do treningowego - czyli w odpowiedzi na sekwencję znaków wyciętą z pewnego miejsca ze zbioru danych, powinien wygenerować sekwencję wyglądającą jak ciąg kolejnych (o 1 miejsce dalszych) znaków.

In [71]:
def train_LSTM(dataset, sequence_size, num_to_char, char_to_num):
    """
    Funkcja wykonująca trening naszej rekurencyjnej sieci neuronowej klasy RNN. Argumenty: \n
    - dataset - torch.tensor zawierający zakodowane (w formie indeksów, int) znaki tekstu treningowego, \n
    - sequence_size - maksymalna długość generowanej sekwencji znaków, int, skalar, \n
    - num_to_char - słownik, wiążący indeks znaku (int) (np. 0) z właściwym znakiem (np.  "a"), len=len(characters), \n
    - char_to_num - słownik, wiążący symbol znaku (np. "a") z przypisanym mu numerem int (np. 0), len=len(characters). \n
    Zwraca: rnn - wytrenowana sieć (obiekt klasy RNN).
    """
    rnn = RNN( # tworzenie obiektu klasy RNN z hidden state o wielkości 512
        input_size=len(num_to_char), 
        hidden_size=512, 
        output_size=len(num_to_char))
    rnn = rnn.double()
    # ----------------------------------- UZUPEŁNIJ KOD ----------------------------------------
    # Utwórz obiekt optimizer, odwołujący się do algorytmu optymalizacji Adam
    optimizer = torch.optim.Adam(rnn.parameters(), lr=0.002)
    # Utwórz obiekt criterion, odwołujący się do funkcji kosztu CrossEntropyLoss
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(700): # trening wykonaj w 700 iteracjach
        rnn.train() # przełączenie sieci w tryb treningu
        start_idx = np.random.randint(dataset.shape[0]-sequence_size) # wygeneruj losowy początek sekwencji treningowej
        # Wyodrębnij z dataset sekwencję treningową - o początku w start_idx i długości sequence_size
        input_sequence = dataset[start_idx : start_idx+sequence_size]
        # Wyodrębnij z dataset sekwencję "walidacyjną" (do liczenia kosztu) - o 1 przesuniętą w stosunku do input_sequence
        val_sequence = dataset[start_idx+1 : start_idx+sequence_size+1]
        hidden_state = None # inicjalizacja hidden state samymi zerami
        # Forward pass - przekaż do sieci neuronowej input_sequence i zbierz predykcję i zawartość hidden_state
        pred, hidden_state = rnn(input_sequence, hidden_state)
        loss = criterion(torch.squeeze(pred), torch.squeeze(val_sequence)) # oblicz wartość kosztu dla tej iteracji
        # Wyzeruj gradienty
        optimizer.zero_grad()
        # Przeprowadź propagację wsteczną kosztu
        loss.backward()
        # Wykonaj 1 iterację algorytmu optymalizacji
        optimizer.step()
        # -------------------------------------------------------------------------------------
        if (epoch%100==0): # przy co 100 iteracji,
            print("Epoch: "+str(epoch)+" , cumulative loss: "+str(loss)) # wyświetl wartość kosztu 
            output_sequence = generate_text_embedding(sequence_size, rnn, char_to_num)
            print(embedding_to_string(output_sequence, num_to_char)) # oraz przykładowy wygenerowany tekst
    return rnn

Uruchom teraz poniższy kod, aby dokonać treningu naszej sieci! Po każdej 100-ej iteracji, wyświetlać Ci się będzie aktualna wartość funkcji kosztu oraz przykładowa sekwencja generowanych przez RNN znaków - zobacz, jak od totalnie losowego ciągu znaków, coraz bardziej upodobniają się one do łacińskich nazw roślin!

In [72]:
# ------------- Trening sieci ---------------
sequence_size = 150
rnn = train_LSTM(
    dataset=embedded_text, 
    sequence_size=sequence_size, 
    num_to_char=num_to_char,
    char_to_num=char_to_num)

Epoch: 0 , cumulative loss: tensor(3.3966, dtype=torch.float64, grad_fn=<NllLossBackward0>)
n
obtcjpqyxiuirqm.lzzo. o
Epoch: 100 , cumulative loss: tensor(2.7075, dtype=torch.float64, grad_fn=<NllLossBackward0>)
oollaycis
 envhtdui tnionnotallasosis rcbla kuiroa aneaenanosnoalm
Epoch: 200 , cumulative loss: tensor(2.4782, dtype=torch.float64, grad_fn=<NllLossBackward0>)
clapaicus dor.pilim
coruetruma
Epoch: 300 , cumulative loss: tensor(2.1540, dtype=torch.float64, grad_fn=<NllLossBackward0>)
lskomhilis nevemi
carifina plgatilhis
Epoch: 400 , cumulative loss: tensor(2.0962, dtype=torch.float64, grad_fn=<NllLossBackward0>)
garokorifa astuaroagices
eprhosium tentiischys
Epoch: 500 , cumulative loss: tensor(2.0343, dtype=torch.float64, grad_fn=<NllLossBackward0>)
guuori camcimenmethlis
flezyuladia
Epoch: 600 , cumulative loss: tensor(2.1508, dtype=torch.float64, grad_fn=<NllLossBackward0>)
nocaethium  lephysmroceara
glunustum cerbistifolia


Na sam koniec, wygeneruj ostateczne nazwy, jakie potrafi utworzyć nasz model.

In [73]:
# ----------------- Predykcja ---------------
# Generacja sekwencji w postaci embeddingów
generated_embeddings = generate_text_embedding(sequence_size, rnn, char_to_num)
# Zamień embeddingi na tekst
generated_text = embedding_to_string(
    sequence_embeddings=generated_embeddings, 
    num_to_char=num_to_char)
# Wyświetl wygenerowany tekst
print(generated_text)

aeinepomoto mawetatus
dybonticha cenoidis


Brawo, umiesz już dokonywać generacji prostego tekstu z wykorzystaniem rekurencyjnych sieci neuronowych!


## 5. Pytania kontrolne
1. Na czym polega działanie rekurencyjnych sieci neuronowych?
2. Wymień kilka rodzajów problemów, jakie można rozwiązać z wykorzystaniem RNN.
3. Czym różni się LSTM od "klasycznej" sieci RNN?

1. **RNN** - sieć neuronowa, która dokonuje predykcji nie tylko na podstawie „obecnych” danych, ale też „wcześniejszych”, np. xt−1 (a dokładniej mówiąc, od pewnego ukrytego stanu ht−1 zależącego od xt−1).
2. **Jakie problemy można rozwiązać wykorzystując RNN (przetwarzanie języka naturalnego - NLP)**:
- tłumaczenie zdań z jednego języka w drugi,
- uzupełnianie brakujących słów/liter w tekście, generacja tekstu,
- rozpoznawanie mowy i tekstu (OCR),
- streszczanie długich tekstów, wyszukiwanie sentymentu.
- analiza sekwencji danych (i zwracać pojedynczą wartość, bądź inną sekwencję, bądź generować nowe sekwencje)
3. **LSTM:**
- wprowadzają mechanizm pamięci długotrwałej
- ułatwia analizę długich sekwencji
- umożliwia zrównoleglenie obliczeń
- wprowadzają dodatkową pamięć ct przechowująca szerszy kontekst, tak długo jak będzie potrzebny
- wprowadza dodatkowe bramki - bramka wejścia, zapomnienia, wyjścia. Pozwalają na kontrolowanie przepływu informacji w sieci: 
  - update gate - jak silnie opierać się na obecnych danych podczas uaktualniania hidden state
  - forget gate - wpływają na to które dane z przeszłości zapomnieć
  - output gate - jak bardzo opierać się na komórce ct podczas uaktualnienia ht.
- tutaj wprowadzono koncepcję komórki przechowywującej informację z poprzednich kroków czasowych
